In [6]:
import os
import numpy as np
import itertools
from typing import List, Tuple
from tensorflow.python.ops.linalg.linear_operator import LinearOperator
from tensorflow.python.framework.ops import EagerTensor
import tensorflow as tf

In [7]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"


In [3]:
def set_diag_vector(diagonal_length: int) -> EagerTensor:
    if diagonal_length <= 1:
        raise ValueError("diagonal_length must be positive and greater than one.")
    if diagonal_length == 1:
         return tf.constant([1], dtype=tf.complex64)
    return tf.constant([1] + [0] * (diagonal_length - 1), dtype=tf.complex64)


In [16]:
def set_on_off_measurement_one_outcome(outcome: int, cutoff_dim: int, diag_vector: EagerTensor = None) -> LinearOperator:
    if outcome != 0 and outcome != 1:
        raise ValueError("outcome must be either 0 or 1")
    if diag_vector is None:
        diag_vector = set_diag_vector(diagonal_length=cutoff_dim + 1)
    if outcome == 0:
        return tf.linalg.LinearOperatorDiag(diag_vector)
    return tf.linalg.LinearOperatorFullMatrix(
        tf.linalg.LinearOperatorIdentity(num_rows=cutoff_dim+1, dtype=tf.complex64).to_dense() -
        tf.linalg.LinearOperatorDiag(diag_vector).to_dense())
    

In [17]:
zero_op = set_on_off_measurement_one_outcome(outcome=0, cutoff_dim=2)


In [18]:
zero_op.to_dense()

<tf.Tensor: shape=(3, 3), dtype=complex64, numpy=
array([[1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j]], dtype=complex64)>

In [12]:
one_op = set_on_off_measurement_one_outcome(outcome=1, cutoff_dim=2)


In [13]:
one_op.to_dense()

<tf.Tensor: shape=(3, 3), dtype=complex64, numpy=
array([[0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j]], dtype=complex64)>

In [19]:
def generate_all_outcomes(modes=int) -> List[Tuple[int]]:
    options = [0, 1]
    return list(itertools.product(options, repeat=modes))


In [20]:
generate_all_outcomes(modes=2)

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [21]:
def compute_kronecker_matrices(matrices: List[LinearOperator]) -> LinearOperator:
    if len(matrices) < 1:
        raise ValueError("The number of matrices must be at least 1")
    return tf.linalg.LinearOperatorKronecker(matrices)


In [28]:
def generate_on_off_measurement_one_outcome(one_outcome: Tuple[int], cutoff_dim: int, diag_vector: List[int] = None) -> LinearOperator:
    if diag_vector is None:
        diag_vector = set_diag_vector(diagonal_length=cutoff_dim + 1)
    one_mode_measurements = [set_on_off_measurement_one_outcome(
        outcome=outcome_i, cutoff_dim=cutoff_dim, diag_vector=diag_vector) for outcome_i in one_outcome]
    return compute_kronecker_matrices(matrices=one_mode_measurements)
    
    

In [29]:
def generate_all_on_off_measurements(modes: int, cutoff_dim: int) -> List[LinearOperator]:
    outcomes = generate_all_outcomes(modes=modes)
    diag_vector = set_diag_vector(diagonal_length=cutoff_dim + 1)
    
    return [generate_on_off_measurement_one_outcome(one_outcome=outcome, cutoff_dim=cutoff_dim, diag_vector=diag_vector) 
            for outcome in outcomes]


In [32]:
ops_measurements = generate_all_on_off_measurements(modes=2, cutoff_dim=2)

In [33]:
[op_measurement.to_dense() for op_measurement in ops_measurements]

[<tf.Tensor: shape=(9, 9), dtype=complex64, numpy=
 array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j]], dtype=complex64)>,
 <tf.Tensor: shape=(9, 9), dtype=complex64, numpy=
 array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.

In [507]:
import os
import strawberryfields as sf
from strawberryfields.backends.tfbackend.states import FockStateTF

In [563]:
alphas = [0.7]
betas = [0.3]
betas = list(np.arange(-8.0, 2.0, 0.1))
modes = 2
cutoff_dim = 2

In [564]:
eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": cutoff_dim})
prog = sf.Program(modes)

tf_alpha = tf.Variable(0.7)
tf_beta = tf.Variable(0.3)

par_alpha = prog.params("alpha")
par_beta = prog.params("beta")

with prog.context as q:
    sf.ops.Dgate(par_alpha) | q[0]
    sf.ops.Dgate(par_beta) | q[1]
    sf.ops.BSgate() | (q[0], q[1])


In [565]:
result = eng.run(prog, args={
    "alpha": tf_alpha,
    "beta": tf_beta,
})

In [568]:
ops_measurements = generate_all_on_off_measurements(modes=modes, cutoff_dim=cutoff_dim)


In [576]:
ops_measurements[1].to_dense()

<tf.Tensor: shape=(4, 4), dtype=complex64, numpy=
array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]], dtype=complex64)>

In [574]:
def compute_probability_all_outcomes(state: FockStateTF, ops_measurements: List[LinearOperator]) -> List[EagerTensor]:
    ket_op = tf.linalg.LinearOperatorFullMatrix(state.data)
    bra_op = tf.linalg.LinearOperatorAdjoint(ket_op)
    dm_op = tf.linalg.LinearOperatorKronecker([ket_op, bra_op])
    print(dm_op.to_dense())
    return [tf.cast(op_measurement.matmul(dm_op).trace(), dtype=tf.float32) for op_measurement in ops_measurements]


In [575]:
compute_probability_all_outcomes(state=result.state, ops_measurements=ops_measurements)


tf.Tensor(
[[0.5598983 +0.j 0.15836315+0.j 0.39590788+0.j 0.11197966+0.j]
 [0.39590788+0.j 0.        +0.j 0.27994916+0.j 0.        +0.j]
 [0.15836315+0.j 0.04479186+0.j 0.        +0.j 0.        +0.j]
 [0.11197966+0.j 0.        +0.j 0.        +0.j 0.        +0.j]], shape=(4, 4), dtype=complex64)


[<tf.Tensor: shape=(), dtype=float32, numpy=0.5598983>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>]

In [435]:
tf.reduce_sum(compute_probability_all_outcomes(state=result.state, ops_measurements=ops_measurements))


<tf.Tensor: shape=(), dtype=float32, numpy=0.5598983>

In [571]:
result.state.fock_prob([0,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.55989826>

In [434]:
tf.reduce_sum([result.state.fock_prob([0,0]),
              result.state.fock_prob([0,1]),
              result.state.fock_prob([1,0]),
              result.state.fock_prob([1,1])])

<tf.Tensor: shape=(), dtype=float32, numpy=0.88463926>

In [577]:
result.state.dm()

<tf.Tensor: shape=(2, 2, 2, 2), dtype=complex64, numpy=
array([[[[0.5598983 +0.j, 0.39590788+0.j],
         [0.39590788+0.j, 0.27994916+0.j]],

        [[0.15836315+0.j, 0.        +0.j],
         [0.11197966+0.j, 0.        +0.j]]],


       [[[0.15836315+0.j, 0.11197966+0.j],
         [0.        +0.j, 0.        +0.j]],

        [[0.04479186+0.j, 0.        +0.j],
         [0.        +0.j, 0.        +0.j]]]], dtype=complex64)>